In [1]:
from common import *

# importing dependencies and load data.
data = load_data_from_cvs("estadao_noticias_eleicao.csv")

In [2]:
data = data.fillna('')

In [3]:
# joyning column of title and sub-title of artile with column of content.
data['articles'] = data['titulo']  + ' ' + data['subTitulo'] + ' ' + data['conteudo']

In [4]:
# normalizing and tokenizing articles.
data['articles'] = data['articles'].apply(normalize)
data['tokens'] = data['articles'].apply(tokenize)

In [5]:
# applying summarization to articles to produce inverted index.
inverted_index = summarize(data['tokens'], data['idNoticia'])

In [6]:
binary_result = search_with_vectorial_model('segundo turno', inverted_index)

In [7]:
tf_result = search_with_tf('segundo turno', inverted_index)

In [8]:
tfidf_result = search_with_tf_idf('segundo turno', inverted_index)

In [9]:
bm25_result = search_with_bm25('segundo turno', inverted_index)

In [10]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [11]:
# Para pesquisa anterior, foram comparados os resultados obtidos pelo meu algoritmo 
# e o gabarito contendo resultados do google.
# Foram usados os algoritmos de busca binária classica, busca usado tf e tf/idf e também usando o algoritmo bm25.
# Essas funções foram implementadas no arquivo 'commom.py' dispponiveis no mesmo diretório de notebook 
# a fim de deixar o script mais cofado em demonstrar a execução e a comparação das respostas com o gabarito do google.

In [12]:
import pandas
import ast

gabarito = pandas.read_csv('gabarito.csv')

to_array = lambda text: ast.literal_eval(text)

# parsing string to array
gabarito.busca_binaria = gabarito.busca_binaria.apply(to_array)
gabarito.tf = gabarito.tf.apply(to_array)
gabarito.tfidf = gabarito.tfidf.apply(to_array)
gabarito.bm25 = gabarito.bm25.apply(to_array)


# create a dataframe with results
suaresposta = {'str_busca': 'segundo turno', 'busca_binaria': [binary_result], 'tf': [tf_result],
              'tfidf': [tfidf_result], 'bm25': [bm25_result]}
suaresposta = pandas.DataFrame(data=suaresposta)


print("Precisão para consulta 'segundo turno' usando busca binaria %.2f%%" % (100 * mapk(gabarito.busca_binaria, suaresposta.busca_binaria, k=5)))

print("Precisão para consulta 'segundo turno' usando busca binaria %.2f%%" % (100 * mapk(gabarito.tf, suaresposta.tf, k=5)))

print("Precisão para consulta 'segundo turno' usando busca binaria %.2f%%" % (100 * mapk(gabarito.tfidf, suaresposta.tfidf, k=5)))

print("Precisão para consulta 'segundo turno' usando busca binaria %.2f%%" % (100 * mapk(gabarito.bm25, suaresposta.bm25, k=5)))

Precisão para consulta 'segundo turno' usando busca binaria 20.00%
Precisão para consulta 'segundo turno' usando busca binaria 80.00%
Precisão para consulta 'segundo turno' usando busca binaria 48.33%
Precisão para consulta 'segundo turno' usando busca binaria 45.33%


In [13]:
# Os resultados das comparações acima mostram que o modelo binario comum se mostra muito simples 
# e por isso não consegue ser muito preciso.

# As funções que usam tf e idf ganham mais precisão por levarem em conta mais fatores e por 
# tentar driblar o problema das stop words no caso da função com tf/idf. Curiosamente, para o
# caso da busca por 'segundo turno' no index o uso de idf fez com que a precisão diminuisce
# se comparado ao gabarito.

# Já para o caso da busca usando o algoritmo bm25, vê-se que a precisão melhora muito, o resultado
# mostrado usa um valor de K = 5 para formula do bm25, que embora esteja foram do range de 1,2--2
# que é o range recomendado se mostrou o valor mais preciso para testes impiricos feitos durante o desenvolvimento.